In [1]:
import os

In [17]:
!rm -rf .zenml
!zenml init

Found existing ZenML repository at path '/Users/nanu/Desktop/ML-Ops/zenml'.
⠋ Initializing ZenML repository at /Users/nanu/Desktop/ML-Ops/zenml.
⠋ Initializing ZenML repository at /Users/nanu/Desktop/ML-Ops/zenml.



In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.base import ClassifierMixin

In [19]:
def train_model() -> None:
    """Train a logistic regression model on the digits dataset."""
    digits = load_digits()
    X, y = digits.data, digits.target
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model= LogisticRegression(penalty='l2', C=0.7, max_iter=1000)
    model.fit(X_train, y_train)
    print(f"Model trained with accuracy: {model.score(X_test, y_test)}")

train_model()

Model trained with accuracy: 0.975


In [28]:
# Import data -> Train model -> Evaluate

from zenml import step, pipeline
from typing_extensions import Annotated
from typing import Any, Dict, Tuple

@step
def load_data() -> Tuple[
                            Annotated[np.ndarray, "X_train"],
                            Annotated[np.ndarray, "X_test"],
                            Annotated[np.ndarray, "y_train"],
                            Annotated[np.ndarray, "y_test"],
                            ]:
    """Loads digits dataset and splits it into train and test sets."""
    digits = load_digits()
    X, y = digits.data, digits.target
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test

@step
def logistic_regression_trainer(X_train: np.ndarray, y_train: np.ndarray) -> ClassifierMixin:
    """Trains a logistic regression model."""
    model= LogisticRegression(penalty='l2', C=0.3, max_iter=1000)
    model.fit(X_train, y_train)
    return model

@step
def evaluator(X_test: np.ndarray, y_test: np.ndarray, model: ClassifierMixin) -> float:
    """Evaluates the sklearn model."""
    accuracy = model.score(X_test, y_test)
    print(f"Model trained with accuracy: {accuracy}")
    return accuracy


In [26]:
@pipeline
def ml_pipeline():
    """Pipeline to load data, train and evaluate a model."""
    X_train, X_test, y_train, y_test = load_data()
    model = logistic_regression_trainer(X_train, y_train)
    score = evaluator(X_test, y_test, model)


In [ ]:
ml_pipeline()

Initiating a new run for the pipeline: ml_pipeline.
Using user: default
Using stack: default
  artifact_store: default
  orchestrator: default
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml login --local.
Using cached version of step load_data.
Using cached version of step logistic_regression_trainer.
Using cached version of step evaluator.
All steps of the pipeline run were cached.


PipelineRunResponse(body=PipelineRunResponseBody(created=datetime.datetime(2025, 4, 17, 19, 46, 30, 742524), updated=datetime.datetime(2025, 4, 17, 19, 46, 31, 395339), user=UserResponse(body=UserResponseBody(created=datetime.datetime(2025, 4, 17, 18, 55, 52, 776415), updated=datetime.datetime(2025, 4, 17, 19, 37, 47, 805874), active=True, activation_token=None, full_name='default', email_opted_in=False, is_service_account=False, is_admin=True, default_project_id=None), metadata=None, resources=None, id=UUID('fab319a1-b537-4799-9fc1-4daa645a45ea'), permission_denied=False, name='default'), status=<ExecutionStatus.COMPLETED: 'completed'>, stack=StackResponse(body=StackResponseBody(created=datetime.datetime(2025, 4, 17, 18, 55, 52, 547997), updated=datetime.datetime(2025, 4, 17, 18, 55, 52, 548009), user=None), metadata=None, resources=None, id=UUID('a7ec8096-6e00-4ac4-a398-33e860540213'), permission_denied=False, name='default'), pipeline=PipelineResponse(body=PipelineResponseBody(creat

In [34]:
def min_trips(N,W):
    dp = [float('inf')] * (N + 1)
    dp[0] = 0
    for i in range(1, N + 1):
        for capacity in W:
            if i >= capacity:
                dp[i] = min(dp[i], dp[i - capacity] + 1)
    return dp[N] if dp[N] != float('inf') else -1


N = 21
M = 6
W = [9,8,5,4,3,1]

capacities = []

for i in range(M):
    capacities.append(W[i])
    for j in range(i+1, M):
        capacities.append(W[i] + W[j])

print(capacities)
print(min_trips(N, capacities))

[9, 17, 14, 13, 12, 10, 8, 13, 12, 11, 9, 5, 9, 8, 6, 4, 7, 5, 3, 4, 1]
2
